# Определение порядка шагов на курсе для датасета events_data_test.csv и events_train.csv

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import Counter
import statistics

In [3]:
import warnings
warnings.filterwarnings('ignore')

Загрузим файл users_count, который  содержит список пользователей у которых:
* 198 passed, равное количеству всех шагов курса;  
* 76 correct, равное количеству практических заданий. 

In [4]:
users_count = pd.read_csv('DATA\\users_count.csv')

In [5]:
# удалим лишнее
users_count = users_count.drop('Unnamed: 0', axis=1)

Загрузим данные, отсортируем и отберем строки соответствующие users_count

In [6]:
data = pd.read_csv('DATA\events_all.zip', compression='zip')

In [7]:
data.sort_values(by=['user_id', 'timestamp'], axis=0, inplace=True)

In [8]:
users_count = users_count.merge(data, on='user_id', how='inner')

Составляем список степов для каждого пользователя из users_count и просуммируем их

In [9]:
steps = users_count.groupby('user_id').step_id.unique().to_frame() \
        .reset_index().rename(columns={'step_id': 'steps_list'})

In [10]:
steps['sum_steps'] = np.nan

In [11]:
for i in range(len(steps)):
    for j in range(len(steps.steps_list[i])):
        steps['sum_steps'][i] = np.sum(steps.steps_list[i])

In [12]:
#проверка совпадения всех степов у каждого юзера
assert steps.sum_steps.min() == steps.sum_steps.max()

In [13]:
#словарь {номер шага: список из номеров этого шага по каждому пользователю}
steps_dict = {}

In [14]:
# заполняем steps_dict
for i in range(len(steps)):
    for j in range(len(steps.steps_list[i])):
        if steps.steps_list[i][j] in steps_dict:
            steps_dict[steps.steps_list[i][j]].append(j)
            #number_step = number_step + 1
        else:
            steps_dict[steps.steps_list[i][j]] = [j]

In [15]:
# таблица, в которой будет восстановлен порядок степов на курсе
ideal_steps = pd.DataFrame()

In [16]:
ideal_steps['step_id'] = steps_dict.keys()
ideal_steps['step_numbers'] = steps_dict.values()
ideal_steps['step_number'] = np.nan

In [17]:
# вычисляем "правильный" номер степа
for i in range(len(ideal_steps)):
    if ideal_steps.step_id[i] in steps_dict:
        ideal_steps['step_number'][i] = statistics.mode(steps_dict.get(ideal_steps.step_id[i]))

In [18]:
# уберем лишнее,отсортируем, изменим тип
ideal_steps = ideal_steps.drop('step_numbers', axis=1)

In [19]:
ideal_steps.sort_values(by=['step_number'], axis=0, inplace=True)

In [20]:
ideal_steps = ideal_steps.astype('int')

Сохраним результаты в файл:

In [21]:
ideal_steps.to_csv('DATA\\ideal_steps_events_all.csv', index=False)

In [22]:
ideal_steps

,step_id,step_number
0,32815,0
1,32811,1
2,32929,2
3,32814,3
4,32812,4
...,...,...
193,109765,193
194,34049,194
195,34050,195
196,34051,196


In [23]:
ideal_steps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198 entries, 0 to 197
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   step_id      198 non-null    int32
 1   step_number  198 non-null    int32
dtypes: int32(2)
memory usage: 3.1 KB
